# Exercise: CRM campaign_Final

After testing our system in the notebook `Exercise: CRM campaign_Teste`, we are now ready to build our final system to based on the files of the 7 past days streams, write a script with the top 50 songs per country. We also check the memory RAM used in this system. As we checked in the notebook `Exercise: CRM campaign_Teste` the system works as required. We also have write a file `README` with the information about how should be used the system.

The system bellow is ready to be used :)!

In [ ]:
import os 
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# function to read the daily log file and return a DataFrame with the data

def read_daily_log_file(log_file):
    # Read the daily log file and return a DataFrame containing the data
    df = pd.read_csv(log_file, sep='|', header=None, names=['sng_id', 'user_id', 'country'])
    return df

In [ ]:
# To compute top_songs:

def compute_top_songs(data, output_file):
   
    # Group by 'country' and 'sng_id' and count the streams
    # We are using `size()`, because we want to include missing values whether they exist
    grouped = data.groupby(['country', 'sng_id']).size().reset_index(name='streams')
   
    # Sort the data (top songs per country)
    sorted_grouped = grouped.sort_values(['country', 'streams'], ascending=[True, False])

    # Create a dictionary to store the top_50 songs for each country
    top_50_songs = {}
    for country, group in sorted_grouped.groupby('country'):
        top_50_songs[country] = group.head(50)

    # Write the top 50 songs for each country to the output file
    with open(output_file, 'w') as file:
        for country, group in top_50_songs.items():
            songs_str = ','.join(f"{sng_id}:{streams}" for _, sng_id, streams in group.itertuples(index=False))
            file.write(f"{country}|{songs_str}\n")

In [ ]:
def main(input_folder, output_folder, start_date):
    
    # Get current date
    today = datetime.today()
    
    # Create a list to store the dates of the last 7 days
    date_range = [today - timedelta(days=i) for i in range(1, 8)]

    data_list = []  # List to store data for the past 7 days

    for day in date_range:
        log_file = f"listen-{day.strftime('%Y-%m-%d')}.txt"
        log_file_path = os.path.join(input_folder, log_file)

        if os.path.isfile(log_file_path):
            data = read_daily_log_file(log_file_path)
            data_list.append(data)
            print(f"Processed log file: {log_file}")
        else:
            print(f"Log file not found: {log_file}")
            

    if data_list:
        
        # Concatenate data from the past 7 days into a single DataFrame
        combined_data = pd.concat(data_list, ignore_index=True)
        output_file = os.path.join(output_folder, f"country_top50_{start_date.strftime('%Y-%m-%d')}.txt")
        compute_top_songs(combined_data, output_file)
        print(f"Top 50 Songs for the past 7 days saved in {output_file}")

In [ ]:
# Let's check the time that takes this system

if __name__ == "__main__":
    input_folder = "data"  
    output_folder = "output_top-50-songs"  
    
    main(input_folder, output_folder)